In [1]:
import pandas as pd
import numpy as np
import geopy.distance

# 2. Pre-processing
     
     
## 2.1 Clean and Load the Dataset

In [2]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
poi_df = pd.read_csv('data/poi.csv')

/tmp/ipykernel_5434/4245473066.py:3: DtypeWarning: Columns (7,9,11,13,17,18,19,22,23,24,25,26,28,29,30,32,33,34,35,36,37,38,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,74,77,80,82,83,84,85,86,87,90,91,92,95) have mixed types. Specify dtype option on import or set low_memory=False.
  poi_df = pd.read_csv('data/poi.csv')


In [3]:
poi_df = poi_df[['lat', 'lon']]

In [12]:
lat, lon = train_df.iloc[0].latitude, train_df.iloc[0].longitude

In [7]:
lat = 52.406374
lon = 16.9251681
series = poi_df.apply(lambda x: geopy.distance.distance((lat, lon), (x['lat'], x['lon'])).km, axis=1)
series[series < 20]

Series([], dtype: float64)

In [18]:
series[series < 400]

Series([], dtype: float64)

In [6]:
def compute_distance_km(lat1, lon1, lat2, lon2):
    return geopy.distance.distance((lat1, lon1), (lat2, lon2)).km
lat1, lon1 = 1, 1
lat2, lon2 = 1, 1
compute_distance_km(lat1, lon1, lat2, lon2)

0.0

In [ ]:
def foo(lat, lon, poi_df):
    series = poi_df.apply(lambda x: geopy.distance.distance((lat, lon), (x['lat'], x['lon'])).km, axis=1)
    series.sort_values()

    return

In [6]:
train_df['poi'] = train_df[['lat', 'lon']].apply(lambda x: foo(x['lat'], x['lon'], poi_df)) 

MemoryError: Unable to allocate 1.15 TiB for an array with shape (157912453924,) and data type int64

In [3]:

geopy.distance.geodesic(coords_1, coords_2).km

279.35290160430094

## 2.2 Feature importance analysis

In [ ]:
# create x, y
df = df.drop('url', axis=1, errors='ignore')
x = df.drop('shares', axis=1)
y = df.shares

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# compute feature_importances
rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(x, y)
rf.feature_importances_

In [ ]:
# plot feature_importances
sorted_idx = rf.feature_importances_.argsort()
plt.barh(x.columns[sorted_idx], rf.feature_importances_[sorted_idx])

In [ ]:
# remove features with feature_importances < 0.01
for importance, feature in zip(rf.feature_importances_, x.columns):
    if importance < 0.02:
        x = x.drop(feature, axis=1)

# compute feature_importances again
rf.fit(x, y)
rf.feature_importances_

In [ ]:
# plot feature_importances again
sorted_idx = rf.feature_importances_.argsort()
plt.barh(x.columns[sorted_idx], rf.feature_importances_[sorted_idx])

## Note 2.3
On the entire dataset, a random forest was used to recognize the most relevant features. This enabled the classification of the importance of the characteristics. Due to a large number of features available, it was decided to consider only the most important. As a result, the features less important than 0.02 have been removed. At the end of this operation, there are 18 features left, accounting for roughly one-third of the total. The importance of the remaining features has been recalculated.

<hr>

## 3. Model Selection (up to 8.2 of 11.2  points)
In this part of the challenge you are requested to perform all the necessary steps required in order to design a full fledged classification task on the <b>shares</b> feature.

You are requested to perform the following steps having in mind the following: 

1) the dataset must be properly splitted to perform crossvalidation 

2) when required, features must be properly encoded

3) in order to simplify the problem the target feature can be dicretized <b>(number of classes must be >=5)</b> ;

4) for model selection you are requested to consider: 

- Decision Trees

- Support Vector Machines;

- An ensamble methodology;

- MLPNs.

5) hyper-parameter tuning <b>must</b> be performed and discussed;

6) apply standardizion and normalization when appropriate;

7) remember to use an appropriate evaluation setting (cross-fold etc..)

8) describe the measures adopted for the evaluation and discuss the results;

9) provide a discussion of the model selection, where you describe the differences in terms of performance and explains the root causes;


I create `new_df` which contains only the features chosen to train the models and 'shares'. 
The value `entry_number_step1` represents the number of samples considered at this point in the code.

In [ ]:
new_df = df.copy()
columns = []
for col in df.columns:
    if col not in x.columns and col != 'shares':
        new_df = new_df.drop(col, axis=1, errors='ignore')
    else:
        columns.append(col)
entry_number_step1 = new_df.shape[0]

i plot hist and hist2d for every feature to see the distribution of the data. Some features have distinct outliers.

In [ ]:
for col in new_df.columns:
    get_hist(new_df, col)

with the following script, I eliminate the outliers. I also tried the other way (with the commented scripts) but in the end, I found it better to eliminate the outliers via zscore. It is reasonable to think that the data at our disposal may have some noise. It is better to remove it for better evaluation.

In [ ]:
from scipy.stats import zscore

z_scores = zscore(new_df)
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 5).all(axis=1)
new_df = new_df[filtered_entries]

I rerun the plot of hist and hist2d for each feature noting some differences. Without the outliers, the distributions are better.

In [ ]:
for col in new_df.columns:
    get_hist(new_df, col)

The number of samples eliminated is 1875, and the number of samples remaining is 37638. I believe it is correct to remove this number of samples, thereby removing data noise.

In [ ]:
entry_number_step2 = new_df.shape[0]
print('Samples before dropping outliers: {}'.format(entry_number_step1))
print('Samples after dropping outliers: {}'.format(entry_number_step2))
print('Samples removed:',format(entry_number_step1-entry_number_step2))

Features (`x`) are scaled, and 'shares' (`y`) is discretized.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import KBinsDiscretizer

x = new_df.drop('shares', axis=1)
y = new_df.shares

scaler = MinMaxScaler()
x_scal = scaler.fit_transform(x)

discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='kmeans')
y_disc = discretizer.fit_transform(y.array.reshape(-1, 1))

I then proceed to divide the dataset into `x_train`, `y_train`. `x_test`, `y_test`. I will only use the first two to train the models. and I will use the last two only and exclusively to test the performance of the models.

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_scal, y_disc, shuffle=True, stratify=y_disc, test_size=1/3, random_state=42)

I create the dictionary of models to train

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

models = {
    'DT': DecisionTreeClassifier(random_state=42),
    'SVM': SVC(random_state=42),
    'BOOST': AdaBoostClassifier(random_state=42),
    'RND_FOREST': RandomForestClassifier(random_state=42),
    'MLPN': MLPClassifier(hidden_layer_sizes=(9,4), random_state=42)
}

I create the three useful functions to train and evaluate models.

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

def fit_models(models, x_train, y_train):
    for name in models.keys():
        print('fitting {}'.format(name))
        models[name].fit(x_train, y_train.ravel())
        print('done')
    return
    
def evaluate_models(models, x_test, y_test, x_scal, y_disc):
    for name, model in models.items():
        print()
        print()
        print()
        print('name: {}'.format(name))
        print()
        print('params:')
        print(model.get_params())
        print()
        y_pred = model.predict(x_test)
        report = classification_report(y_test, y_pred, zero_division=0, target_names=['class0', 'class1', 'class2', 'class3', 'class4'])
        cm = confusion_matrix(y_test, y_pred)
        cv_accuracy = cross_val_score(model, x_scal, y_disc.ravel(), n_jobs=-1, scoring='accuracy')
        cv_f1_macro = cross_val_score(model, x_scal, y_disc.ravel(), n_jobs=-1, scoring='f1_macro')
        print('report:')
        print(report)
        print()
        print('confusion_matrix:')
        print(cm)
        print()
        print('cross_val_score (accuracy):')
        print(cv_accuracy)
        print("%0.4f (+/- %0.4f)" % (cv_accuracy.mean(), cv_accuracy.std() * 2))
        print()
        print('cross_val_score (f1_macro):')
        print(cv_f1_macro)
        print("%0.4f (+/- %0.4f)" % (cv_f1_macro.mean(), cv_f1_macro.std() * 2))
        print()
        print()
        print()
    return

Before training the models, I print the dimensions of the train and test dataset. it is useful to stick to a fixed proportion to correctly perform cross-validation.

In [ ]:
train_perc = round((x_train.shape[0]/(x_train.shape[0]+x_test.shape[0]))*100, 2)
print('train set dimention: {} ({}%)'.format(x_train.shape[0], train_perc))
print('test set dimention: {} ({}%)'.format(x_test.shape[0], 100-train_perc))

In [ ]:
fit_models(models, x_train, y_train)

In [ ]:
evaluate_models(models, x_test, y_test, x_scal, y_disc)

summary evaluation:
<table>
    <tr>
        <th> name </th><th>accuracy</th><th>f1macro</th><th>cv_accuracy</th><th>cv_f1macro</th>
    </tr>
    <tr>
        <td>DT</td><td>0.66</td><td>0.21</td><td>0.4071 (+/- 0.3962)</td><td>0.1513 (+/- 0.1054)</td>
    </tr>
    <tr>
        <td>SVM</td><td>0.80</td><td>0.18</td><td>0.7968 (+/- 0.0001)</td><td>0.1774 (+/- 0.0000)</td>
    </tr>
    <tr>
        <td>BOOST</td><td>0.80</td><td>0.18</td><td>0.7957 (+/- 0.0037)</td><td>0.1783 (+/- 0.0023)</td>
    </tr>
    <tr>
        <td>RND_FOREST</td><td>0.80</td><td>0.18</td><td>0.6614 (+/- 0.5165)</td><td>0.1570 (+/- 0.1051)</td>
    </tr>
    <tr>
        <td>MLPN</td><td>0.80</td><td>0.18</td><td>0.7968 (+/- 0.0001)</td><td>0.1774 (+/- 0.0000)</td>
    </tr>
</table>

It was discovered that the unbalanced dataset resulted in excellent accuracy but a very poor f1_macro. This is due to the imbalanced dataset. We want to correctly predict all classes, we cannot predict exactly class 0 only. for this we want to maximize f1_macro while maintaining high accuracy.

We note that the models have been trained with a large amount of class 0 samples.

In [ ]:
plt.hist(y_train, bins=5)

the intent is to balance the classes. target:
- class 0: 1/3 samples
- class 1: 1/4 samples
- class 2: 1/5 samples
- class 3: 1/6 samples
- class 4: 1/7 samples

the distribution totals 1.0928, it is tolerable, we will have just over 2/3 of the training set

In [ ]:
c0 = min(np.count_nonzero(y_train == 0), int((1/3)*y_train.shape[0]))
c1 = min(np.count_nonzero(y_train == 1), int((1/4)*y_train.shape[0]))
c2 = min(np.count_nonzero(y_train == 2), int((1/5)*y_train.shape[0]))
c3 = min(np.count_nonzero(y_train == 3), int((1/6)*y_train.shape[0]))
c4 = min(np.count_nonzero(y_train == 4), int((1/7)*y_train.shape[0]))

We use RandomUnderSampler to undersampling classes that contain more samples than desired.

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(sampling_strategy={0:c0, 1:c1, 2:c2, 3:c3, 4:c4}, random_state=42)
x_train_under, y_train_under = rus.fit_resample(x_train, y_train)

In [ ]:
x_train_under_dim = x_train_under.shape[0]
print('Samples before undersampling: {}'.format(x_train.shape[0]))
print('Samples after undersampling: {}'.format(x_train_under_dim))
print('Samples removed: {}'.format(x_train.shape[0]-x_train_under_dim))

In [ ]:
plt.hist(y_train_under, bins=5)

We use SMOTE to oversampling classes that contain less samples than desired.

In [ ]:
c0 = max(np.count_nonzero(y_train_under == 0), int((1/3)*y_train.shape[0]))
c1 = max(np.count_nonzero(y_train_under == 1), int((1/4)*y_train.shape[0]))
c2 = max(np.count_nonzero(y_train_under == 2), int((1/5)*y_train.shape[0]))
c3 = max(np.count_nonzero(y_train_under == 3), int((1/6)*y_train.shape[0]))
c4 = max(np.count_nonzero(y_train_under == 4), int((1/7)*y_train.shape[0]))

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(k_neighbors=3, n_jobs=-1, sampling_strategy={0:c0,1:c1,2:c2,3:c3,4:c4}, random_state=42)
x_train_over, y_train_over = sm.fit_resample(x_train_under, y_train_under)

In [ ]:
entry_number_step4 = x_train_over.shape[0]
print('Samples before oversampling: {}'.format(x_train_under_dim))
print('Samples after oversampling: {}'.format(entry_number_step4))
print('Samples added: {}'.format(entry_number_step4-x_train_under_dim))

Now the samples are balanced as originally planned

In [ ]:
plt.hist(y_train_over, bins=5)

I plot the distribution of the features following the new samples

In [ ]:
for i, col in enumerate(columns):
    if col != 'shares':
        to_plot = x_train_over[:,i]
    else:
        to_plot = y_train_over
    min_value = to_plot.min()
    max_value = to_plot.max()
    b = 100
    plt.hist(to_plot, bins=b)
    plt.title('histogram distribution of {} (min_value: {}, max_value: {}, bins: {})'.format(col, min_value, max_value, b))
    plt.show()
    plt.close()
    plt.hist2d(pd.Series(np.array([i for i in range(to_plot.shape[0])])), to_plot, bins=b, norm = colors.LogNorm())
    plt.title('2D histogram distribution of {} (min_value: {}, max_value: {}, bins: {})'.format(col, min_value, max_value, b))
    plt.show()
    plt.close()

In [ ]:
train_perc = round((x_train_over.shape[0]/(x_train_over.shape[0]+x_test.shape[0]))*100, 2)
print('train set dimention: {} ({}%)'.format(x_train_over.shape[0], train_perc))
print('test set dimention: {} ({}%)'.format(x_test.shape[0], 100-train_perc))

I fit and evaluate models with 2/3 of training set and 1/3 of test set

In [ ]:
fit_models(models, x_train_over, y_train_over)

In [ ]:
evaluate_models(models, x_test, y_test, x_scal, y_disc)

summary evaluation:
<table>
    <tr>
        <th> name </th><th>accuracy</th><th>f1macro</th><th>cv_accuracy</th><th>cv_f1macro</th>
    </tr>
    <tr>
        <td>DT</td><td>0.49</td><td>0.20</td><td>0.4071 (+/- 0.3962)</td><td>0.1513 (+/- 0.1054)</td>
    </tr>
    <tr>
        <td>SVM</td><td>0.65</td><td>0.25</td><td>0.7968 (+/- 0.0001)</td><td>0.1774 (+/- 0.0000)</td>
    </tr>
    <tr>
        <td>BOOST</td><td>0.59</td><td>0.23</td><td>0.7957 (+/- 0.0037)</td><td>0.1783 (+/- 0.0023)</td>
    </tr>
    <tr>
        <td>RND_FOREST</td><td>0.70</td><td>0.26</td><td>0.6614 (+/- 0.5165)</td><td>0.1570 (+/- 0.1051)</td>
    </tr>
    <tr>
        <td>MLPN</td><td>0.65</td><td>0.22</td><td>0.7968 (+/- 0.0001)</td><td>0.1774 (+/- 0.0000)</td>
    </tr>
</table>

With under/over sampling a better f1_macro is noted, while maintaining good accuracy. To choose the best model I considered these values in order: cv_f1macro, cv_accuracy, f1_macro, accuracy. I selected SVC as the best model and I proceed with the hyper-parameter tuning.

I define function for model tuning.

In [ ]:
from sklearn.model_selection import GridSearchCV

def tune_model(model, param_grid, scoring, x_train, y_train, grid_jobs):
    print('tuning...')
    clf = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, verbose=1, n_jobs=grid_jobs)
    clf.fit(x_train, y_train.ravel())
    print('done')
    print()
    print("Best: %f using %s" % (clf.best_score_, clf.best_params_))
    best_params = clf.best_params_.copy()
    return best_params

I perform tuning on half of the test dataset, or one-third of the total dataset.

In [ ]:
c0 = min(np.count_nonzero(y_train == 0), int((1/6)*y_train.shape[0]))
c1 = min(np.count_nonzero(y_train == 1), int((1/8)*y_train.shape[0]))
c2 = min(np.count_nonzero(y_train == 2), int((1/10)*y_train.shape[0]))
c3 = min(np.count_nonzero(y_train == 3), int((1/12)*y_train.shape[0]))
c4 = min(np.count_nonzero(y_train == 4), int((1/14)*y_train.shape[0]))

In [ ]:
rus = RandomUnderSampler(sampling_strategy={0:c0, 1:c1, 2:c2, 3:c3, 4:c4}, random_state=42)
x_tuning, y_tuning = rus.fit_resample(x_train, y_train)

In [ ]:
c0 = max(np.count_nonzero(y_tuning == 0), int((1/6)*y_train.shape[0]))
c1 = max(np.count_nonzero(y_tuning == 1), int((1/8)*y_train.shape[0]))
c2 = max(np.count_nonzero(y_tuning == 2), int((1/10)*y_train.shape[0]))
c3 = max(np.count_nonzero(y_tuning == 3), int((1/12)*y_train.shape[0]))
c4 = max(np.count_nonzero(y_tuning == 4), int((1/14)*y_train.shape[0]))

In [ ]:
sm = SMOTE(k_neighbors=3, n_jobs=-1, sampling_strategy={0:c0,1:c1,2:c2,3:c3,4:c4}, random_state=42)
x_tuning, y_tuning = sm.fit_resample(x_tuning, y_tuning)

In [ ]:
plt.hist(y_tuning, bins=5)

I set param grid for tuning model with GridSearchCV

In [ ]:
SVM_param_grid = {
    'C': [0.5, 1],
    'kernel': ['rbf', 'sigmoid'],
    'gamma': ['scale', 'auto'],
    'decision_function_shape': ['ovo', 'ovr'],
    'random_state': [42]
}

I tune model by accuracy and i find `best_accuracy_params`

In [ ]:
best_accuracy_params = tune_model(SVC(), SVM_param_grid, 'accuracy', x_tuning, y_tuning, -1)

I tune model by f1_macro and i find `best_f1macro_params`

In [ ]:
best_f1macro_params = tune_model(SVC(), SVM_param_grid, 'f1_macro',  x_tuning, y_tuning, -1)

I fit and evaluate the two models with the best parameters found

In [ ]:
tuned_models = {
    'SVM_accuracy': SVC(**best_accuracy_params),
    'SVM_f1macro': SVC(**best_f1macro_params)
}

In [ ]:
fit_models(tuned_models, x_train_over, y_train_over)

In [ ]:
evaluate_models(tuned_models, x_test, y_test, x_scal, y_disc)

## Note 3
The outliers were removed from the entire dataset using zscore, assuming there was noise in the dataset. The dataset has been divided into two sections: train (66%) and test (33%). Five distinct models were trained, tested, and evaluated. It was discovered that the unbalanced dataset resulted in excellent accuracy but a very poor f1_macro. 
The larger classes were undersampled, while the smaller classes were oversampled. The new class samples distribution is: c0 = 1/3, c1 = 1/4, c2 = 1/5, c3 = 1/6, c4 = 1/7.
This division still keeps the classes unbalanced, but it is much less severe than it was at the start. Furthermore, I think that this sample division is more useful for solving the problem at hand than having the same number of samples for all classes. This is because, given the nature of the problem (prediction of the share of articles), I think that the first classes will be far more populated than the last classes. Non-viral articles outnumber viral articles.
With the new dataset, models were trained, tested, and evaluated. These demonstrate a significant improvement in f1_macro. SVC was chosen as the best model and it was tuned. Because of the computational complexity of SVC, the tuning was done on a smaller grid than I would have preferred.
The SVM with the following parameters {'C': 1, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovo', 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': 42, 'shrinking': True, 'tol': 0.001, 'verbose': False} is the best model for solving the problem.

<hr>

# 4. Summary
Provide a summary discussion (in English) of your solution <b>(at least 500 words)</b> feel free to include plots figures and tables.

<b>This is a mandatory step</b>

The solution for forecasting article share is discussed in detail below. The dataset was loaded first, and then the rows with at least one of the following characteristics were deleted:
- number of values other than the number of columns;
- None values;
- n/a. the values;
- values outside the domain of the respective column.

This operation removed 135 lines, resulting in a clean dataset of 39513 samples.
As a result, the feature importance has been investigated using a RandomForestRegressor. As a result, features with an importance less than 0.02 have been removed. The remaining features after this operation were 18.

Graphs were created to investigate the value distribution for each feature. Because of the non-homogeneous distribution, 1875 samples were eliminated as outliers. Because it was assumed that the dataset at our disposal could not be perfect, some noise was removed. To avoid losing important data, not too many samples were deleted.
The final dataset contains 37638 samples and 18 features. This was divided into train sets (66%) and test sets (33%).

Models have been developed, trained, and evaluated. The results show that the accuracy is excellent, but the f1 macro is low. Almost all models classify the entirety of the samples as class0, and because class0 is the majority class, the accuracy is high; however, we cannot create models that classify everything in a single class. We must develop models that are sensitive to the classification of all five classes.

For studying predictions on unbalanced classes, f1 macro is a useful value. Our problem has unbalanced classes: it is more likely that an article will be little shared than that it will go viral, and there are many less famous articles than famous articles. Because the nature of the problem requires unbalanced classes, our goal is to have greater sensitivity for the majority classes while also being able to predict the other classes. As a result, we want to increase f1_macro while maintaining accuracy.
to do this, the train set is balanced as follows:
- class 0: 1/3 samples
- class 1: 1/4 samples
- class 2: 1/5 samples
- class 3: 1/6 samples
- class 4: 1/7 samples

the distribution totals 1.0928, it is tolerable, we will have just over 2/3 of the training set.

An uniform sample distribution was also tested, but in the evaluation, poor results were obtained in the majority classes, precisely because the problem has unbalanced classes by definition, but models with balanced classes were trained. Another attempt was to distribute the test set for 4/7 on the first two classes and 3/7 on the last three classes, but this resulted in a correct prediction of the first two classes and a bad prediction of the last three, reducing the problem to the prediction of only two classes. As a result, the solution's choice turned out to be the best.

The models were trained and tested obtaining the following results:
<table>
    <tr>
        <th> name </th><th>accuracy</th><th>f1macro</th><th>cv_accuracy</th><th>cv_f1macro</th>
    </tr>
    <tr>
        <td>DT</td><td>0.49</td><td>0.20</td><td>0.4071 (+/- 0.3962)</td><td>0.1513 (+/- 0.1054)</td>
    </tr>
    <tr>
        <td>SVM</td><td>0.65</td><td>0.25</td><td>0.7968 (+/- 0.0001)</td><td>0.1774 (+/- 0.0000)</td>
    </tr>
    <tr>
        <td>BOOST</td><td>0.59</td><td>0.23</td><td>0.7957 (+/- 0.0037)</td><td>0.1783 (+/- 0.0023)</td>
    </tr>
    <tr>
        <td>RND_FOREST</td><td>0.70</td><td>0.26</td><td>0.6614 (+/- 0.5165)</td><td>0.1570 (+/- 0.1051)</td>
    </tr>
    <tr>
        <td>MLPN</td><td>0.65</td><td>0.22</td><td>0.7968 (+/- 0.0001)</td><td>0.1774 (+/- 0.0000)</td>
    </tr>
</table>

Due to the obvious presence of better models, DT and BOOST were discarded. RND FOREST performed well on the test set but was rejected due to insufficient crossvalidation results (the crossvalidation values had a greater weight in my evaluation). Both the MLPN and SVM models have been fine-tuned (with half of the test set). Model overfitting was caused by MLPN tuning, whereas SVM tuning did not underperform or outperform the model results prior to tuning. Due to MLPN's overfitting flaw, SVM was presented as the best model ('SVM_accuracy' in `tuned_models`).

It should be noted that the test set was only used to evaluate the model and was not altered or balanced in any way. If the test set had been balanced in the same way that the train set was, I believe the results would have been significantly better. However, this behavior would have compromised the solution's integrity. It is also important to note that the train set after sample redistribution remains 2/3 of the total set, allowing for a correct crossvalidation result.